In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0
# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)
# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
  perturbation = epsilon * np.sign(gradient)
  adversarial_image = image + perturbation
  adversarial_image = np.clip(adversarial_image, 0, 1)
  return adversarial_image
# Функция для генерации противоречивых примеров
def generate_adversarial_examples(model, images, labels, epsilon):
  adversarial_images = []
  for i in range(len(images)):
    image = tf.convert_to_tensor(images[i].reshape((1, 28, 28)))
    label = labels[i]
    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label),
    label = tf.convert_to_tensor(label)
    with tf.GradientTape() as tape:
      tape.watch(image)
      prediction = model(image)
      loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)
    gradient = tape.gradient(loss, image)
    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
    adversarial_images.append(adversarial_image.reshape(28, 28))
  return np.array(adversarial_images)
# Создание модели
def create_model():
  model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model
# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
 for epoch in range(5): # Короткое обучение для демонстрации
  print(f'Epoch:{epoch}')
  for i in range(0, len(train_images), 64):
    batch_images = train_images[i:i+64]
    batch_labels = train_labels[i:i+64]
    # Генерация противоречивых примеров для текущей партии данных
    adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
    # Объединение оригинальных и противоречивых примеров
    combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
    combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
    # Обучение на комбинированных данных
    model.train_on_batch(combined_images, combined_labels)
# Инициализация модели
model = create_model()
# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images[:1500], train_labels[:1500], epsilon=0.1) # ограничил тренировочные данные для ускорения обучения
# Сохранение защищенной модели
model.save('adversarial_trained_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch:0
Epoch:1
Epoch:2
Epoch:3
Epoch:4


In [9]:
model.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8413 - loss: 0.5702


[0.4851238429546356, 0.8680999875068665]

In [8]:
# Реализация градиентной маскировки
# Для демонстрации мы можем использовать специальные функции активации
from tensorflow.keras.layers import Activation
# Обновление модели для градиентной маскировки
def create_masked_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu'),
 tf.keras.layers.Dense(10), Activation('softplus') # Используем softplus вместо softmax для градиентной маскировки
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
 return model
# Обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с градиентной маскировкой
masked_model.save('masked_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8723 - loss: 0.4679
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9637 - loss: 0.1214
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9757 - loss: 0.0812
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9814 - loss: 0.0625
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9856 - loss: 0.0486


In [10]:
# Модель с регуляризацией и нормализацией
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),
 tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])
 return model
# Обучение модели с регуляризацией и нормализацией
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с регуляризацией
regularized_model.save('regularized_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7953 - loss: 1.3433
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8688 - loss: 0.6323
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8608 - loss: 0.6335
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8634 - loss: 0.6234
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8611 - loss: 0.6126


In [12]:
# Загрузка атакованной модели
protected_model = tf.keras.models.load_model('adversarial_trained_model.h5')
# Генерация противоречивых примеров для тестовых данных
adversarial_test_images = generate_adversarial_examples(protected_model, test_images[1000:2000], test_labels[1000:2000], epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = protected_model.evaluate(adversarial_test_images, test_labels[1000:2000])
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.4243 - loss: 1.6112
Accuracy of protected model on adversarial examples: 0.4440000057220459


Модель, обученная на противоречивых примерах, показала плохой результат на незнакомых данных. Противоречивые примеры создавались конкретно для данной модели.

Проверим также её точность на атакованных данных, используемых при обучении

In [14]:
adversarial_test_images = generate_adversarial_examples(protected_model, train_images[:1000], train_labels[:1000], epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = protected_model.evaluate(adversarial_test_images, train_labels[:1000])
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6839 - loss: 1.0497 
Accuracy of protected model on adversarial examples: 0.6610000133514404


Не самая выдающаяся точность. Возможно, можно её повысить, увеличив время обучения и выборку, что, однако, очень требовательно к вычислительным ресурсам

In [20]:
base_model = create_model()
base_model.fit(train_images, train_labels, epochs=5)
# --- Оценка моделей на противоречивых примерах ---
test_adversarial_images = generate_adversarial_examples(base_model, test_images[:3000], test_labels[:3000], epsilon=0.1)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8795 - loss: 0.4271
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9641 - loss: 0.1232
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9771 - loss: 0.0783
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9837 - loss: 0.0553
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9875 - loss: 0.0420


In [16]:
# Оценка защищенных моделей
print("Evaluation of models on adversarial examples:")
print("Base Model Accuracy on Adversarial Examples:")
base_model.evaluate(test_adversarial_images, test_labels[:3000])
print("Adversarially Trained Model Accuracy on Adversarial Examples:")
protected_model.evaluate(test_adversarial_images, test_labels[:3000])
print("Masked Model Accuracy on Adversarial Examples:")
masked_model.evaluate(test_adversarial_images, test_labels[:3000])
print("Regularized Model Accuracy on Adversarial Examples:")
regularized_model.evaluate(test_adversarial_images, test_labels[:3000])

Evaluation of models on adversarial examples:
Base Model Accuracy on Adversarial Examples:
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.1022 - loss: 6.6053
Adversarially Trained Model Accuracy on Adversarial Examples:
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6924 - loss: 0.9503
Masked Model Accuracy on Adversarial Examples:
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2210 - loss: 4.6751
Regularized Model Accuracy on Adversarial Examples:
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6766 - loss: 1.1007


[1.1625213623046875, 0.6543333530426025]

Как видно, модель, обученная на противоречивых примерах, и модель с регуляризацией и нормализацией показали неплохой результат. А модель с маскировкой спраивлась гораздо хуже. Конечно, каждая из моделей справилась лучше, чем модель без защиты, но точность ~65% вряд ли можно назвать удовлетворительной. Впрочем, каждое из изображений в данном тесте было противоречивым, так что можно утверждать, что как минимум половина атак FGSM не повлияют на модели с противоречивым обучением и регуляризацией с нормализацией

Противоречивые примеры были сгенерированы на базовой модели. Посмотрим, как поведут себя модели, если противоречивые примеры будут сгенерированы конкретно для них (вариант для обученной на противоречивых данных модели был показан выше, поэтому будет пропущен)

In [18]:
adversarial_test_images = generate_adversarial_examples(masked_model, train_images[:1000], train_labels[:1000], epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = masked_model.evaluate(adversarial_test_images, train_labels[:1000])
print(f'Accuracy of masked model on adversarial examples: {test_acc}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1349 - loss: 6.1444
Accuracy of masked model on adversarial examples: 0.14499999582767487


In [19]:
adversarial_test_images = generate_adversarial_examples(regularized_model, train_images[:1000], train_labels[:1000], epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = regularized_model.evaluate(adversarial_test_images, train_labels[:1000])
print(f'Accuracy of regularized model on adversarial examples: {test_acc}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4902 - loss: 1.6612
Accuracy of regularized model on adversarial examples: 0.460999995470047


Из данных тестов видно, что при атаке на конкретную модель точность упадёт ещё сильнее, а маскированная модель справляется совсем плохо. Таким образом, наиболее успешными себя показали модель, обученная на противоречивых примерах, и модель с регуляризацией и нормализацией